# Importacion de datos


Por eficiencia, se considero necesario almacenar el dataset en Goggle Drive. Por tanto se tiene que acceder a este en un principio.

In [ ]:
!pip install gdown

In [ ]:
import gdown

In [ ]:
gdown.download( 'https://drive.google.com/u/2/uc?id=1e0_2YcawvoSdS-V6itdkQiqfkCn6AQws&export=download&authuser=0', quiet=False )

Downloading...
From: https://drive.google.com/u/2/uc?id=1e0_2YcawvoSdS-V6itdkQiqfkCn6AQws&export=download&authuser=0
To: /content/CredViv.zip
100%|██████████| 723M/723M [00:09<00:00, 76.5MB/s]


'CredViv.zip'

In [ ]:
%%bash
unzip /content/CredViv.zip

Archive:  /content/CredViv.zip
  inflating: CredViv/application_test.csv  
  inflating: CredViv/application_train.csv  
  inflating: CredViv/bureau.csv      
  inflating: CredViv/bureau_balance.csv  
  inflating: CredViv/credit_card_balance.csv  
  inflating: CredViv/HomeCredit_columns_description.csv  
  inflating: CredViv/installments_payments.csv  
  inflating: CredViv/POS_CASH_balance.csv  
  inflating: CredViv/previous_application.csv  
  inflating: CredViv/sample_submission.csv  


Puesto que el Dataset es demasiado pesado se lo subio como archivo comprimido Zip, a continucaion, se lo descomprime.

# Instalacion de librerias
Instalamos las librerias que usaremos por ahora

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

# Lectura de datos
Empezamos enumerando todos los archivos de datos disponibles. Hay un total de 9 archivos: 1 archivo principal de entrenamiento (con objetivo), 1 archivo principal para pruebas (sin objetivo), 1 archivo de envío de ejemplo y otros 6 archivos que contienen información adicional sobre cada préstamo.

In [ ]:
application_test = pd.read_csv('/content/CredViv/application_test.csv')
bureau = pd.read_csv('/content/CredViv/bureau.csv')
bureau_balance = pd.read_csv('/content/CredViv/bureau_balance.csv')
application_train = pd.read_csv('/content/CredViv/application_train.csv')
POS_CASH_balance = pd.read_csv('/content/CredViv/POS_CASH_balance.csv')
previous_application = pd.read_csv('/content/CredViv/previous_application.csv')
installments_payments = pd.read_csv('/content/CredViv/installments_payments.csv')
credit_card_balance = pd.read_csv('/content/CredViv/credit_card_balance.csv')

Analizamos el tamaño de los datos para tener una idea. A continuacion se muestran las columnas y filas para cada uno.

In [ ]:
print('Size of application_train data', application_train.shape)
print('Size of POS_CASH_balance data', POS_CASH_balance.shape)
print('Size of bureau_balance data', bureau_balance.shape)
print('Size of previous_application data', previous_application.shape)
print('Size of installments_payments data', installments_payments.shape)
print('Size of credit_card_balance data', credit_card_balance.shape)
print('Size of bureau data', bureau.shape)
print('Size of application_test data', application_test.shape)

Size of application_train data (307511, 122)
Size of POS_CASH_balance data (10001358, 8)
Size of bureau_balance data (27299925, 3)
Size of previous_application data (1670214, 37)
Size of installments_payments data (13605401, 8)
Size of credit_card_balance data (3840312, 23)
Size of bureau data (1716428, 17)
Size of application_test data (48744, 121)


Organizamos esta informacion

In [ ]:
data_sizes = {
    'Dataframe': ['application_train', 'POS_CASH_balance', 'bureau_balance', 'previous_application', 'installments_payments', 'credit_card_balance', 'bureau','application_test'],
    'Filas': [307511, 10001358, 27299925, 1670214, 13605401, 3840312, 1716428, 48744],
    'Columnas': [122, 8, 3, 37, 8, 23, 17, 121]
}
df_sizes = pd.DataFrame(data_sizes)
print(df_sizes)

               Dataframe     Filas  Columnas
0      application_train    307511       122
1       POS_CASH_balance  10001358         8
2         bureau_balance  27299925         3
3   previous_application   1670214        37
4  installments_payments  13605401         8
5    credit_card_balance   3840312        23
6                 bureau   1716428        17
7       application_test     48744       121


# Filtrado de información
En el proyecto, nos limitaremos a utilizar solo los datos contenidos en application_train, pues contienen la variable TARGET, que es la que nos piden predecir en la competencia. Esto nos permitirá establecer una base que luego podremos mejorar.

Los datos de entrenamiento tienen 307511 observaciones (cada una es un préstamo separado) y 122 características (variables), incluido el TARGET (la etiqueta que queremos predecir).

In [ ]:
print('Training data shape: ', application_train.shape)
application_train.head()

Training data shape:  (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


Veamos las variables categoricas y numericas

In [ ]:
application_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


Vamos a determinar que columnas categoricas tienen un solo subnivel, con la finalidad de dscartar esta informacion que no nos aporta

In [ ]:
columnas_categoricas = application_train.select_dtypes(include=['object']).columns.tolist()

for columna in columnas_categoricas:
    num_subniveles = application_train[columna].nunique()
    print(f'Columna "{columna}" tiene {num_subniveles} subniveles únicos.')


Columna "NAME_CONTRACT_TYPE" tiene 2 subniveles únicos.
Columna "CODE_GENDER" tiene 3 subniveles únicos.
Columna "FLAG_OWN_CAR" tiene 2 subniveles únicos.
Columna "FLAG_OWN_REALTY" tiene 2 subniveles únicos.
Columna "NAME_TYPE_SUITE" tiene 7 subniveles únicos.
Columna "NAME_INCOME_TYPE" tiene 8 subniveles únicos.
Columna "NAME_EDUCATION_TYPE" tiene 5 subniveles únicos.
Columna "NAME_FAMILY_STATUS" tiene 6 subniveles únicos.
Columna "NAME_HOUSING_TYPE" tiene 6 subniveles únicos.
Columna "OCCUPATION_TYPE" tiene 18 subniveles únicos.
Columna "WEEKDAY_APPR_PROCESS_START" tiene 7 subniveles únicos.
Columna "ORGANIZATION_TYPE" tiene 58 subniveles únicos.
Columna "FONDKAPREMONT_MODE" tiene 4 subniveles únicos.
Columna "HOUSETYPE_MODE" tiene 3 subniveles únicos.
Columna "WALLSMATERIAL_MODE" tiene 7 subniveles únicos.
Columna "EMERGENCYSTATE_MODE" tiene 2 subniveles únicos.


Podemos observar que ninguna tiene un solo subnivel, por la cual no se la puede descartar

Hacemos lo mismo para los datos numericos, donde tampoco podemos descartar alguno

In [ ]:
# Selecciona todas las columnas numéricas
columnas_numericas = application_train.select_dtypes(include='number')

# Calcula la desviación estándar para cada columna numérica y muestra los resultados
std_por_columna = columnas_numericas.std()
columnas_con_desviacion_cero = std_por_columna[std_por_columna == 0].index.tolist()

print("Columnas con desviación estándar igual a cero:")
for columna in columnas_con_desviacion_cero:
    print(columna)

Columnas con desviación estándar igual a cero:


Aplicamos un codigo que nos permita filtrar las filas repetidas.

In [ ]:
print(f'Tamaño del set antes de eliminar las filas repetidas: {application_train.shape}')
application_train.drop_duplicates(inplace=True)
print(f'Tamaño del set después de eliminar las filas repetidas: {application_train.shape}')

Tamaño del set antes de eliminar las filas repetidas: (307511, 122)
Tamaño del set después de eliminar las filas repetidas: (307511, 122)


Como podemos observar, conservamos el mismo numero de filas, pues no se repite ninguna.

A continuacion, vamos a analizar los outliers, a partir de un botplox que nos permitira visualizar y analizar estos valores extremos numericos

In [ ]:
datos_ordenados = [
    'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_INCOME_TOTAL', 'AMT_REQ_CREDIT_BUREAU_DAY',
    'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
    'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'APARTMENTS_AVG', 'APARTMENTS_MEDI',
    'APARTMENTS_MODE', 'BASEMENTAREA_AVG', 'BASEMENTAREA_MEDI', 'BASEMENTAREA_MODE', 'CNT_CHILDREN',
    'CNT_FAM_MEMBERS', 'COMMONAREA_AVG', 'COMMONAREA_MEDI', 'COMMONAREA_MODE', 'DAYS_BIRTH',
    'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_REGISTRATION',
    'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'ELEVATORS_AVG', 'ELEVATORS_MEDI',
    'ELEVATORS_MODE', 'ENTRANCES_AVG', 'ENTRANCES_MEDI', 'ENTRANCES_MODE', 'EXT_SOURCE_1',
    'EXT_SOURCE_2', 'EXT_SOURCE_3', 'FLAG_CONT_MOBILE', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
    'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16',
    'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_20',
    'FLAG_DOCUMENT_21', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
    'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_EMAIL', 'FLAG_EMP_PHONE', 'FLAG_MOBIL',
    'FLAG_PHONE', 'FLAG_WORK_PHONE', 'FLOORSMAX_AVG', 'FLOORSMAX_MEDI', 'FLOORSMAX_MODE', 'FLOORSMIN_AVG',
    'FLOORSMIN_MEDI', 'FLOORSMIN_MODE', 'HOUR_APPR_PROCESS_START', 'LANDAREA_AVG', 'LANDAREA_MEDI',
    'LANDAREA_MODE', 'LIVE_CITY_NOT_WORK_CITY', 'LIVE_REGION_NOT_WORK_REGION', 'LIVINGAPARTMENTS_AVG',
    'LIVINGAPARTMENTS_MEDI', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_AVG', 'LIVINGAREA_MEDI', 'LIVINGAREA_MODE',
    'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_AVG',
    'NONLIVINGAREA_MEDI', 'NONLIVINGAREA_MODE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
    'OWN_CAR_AGE', 'REGION_POPULATION_RELATIVE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
    'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'REG_REGION_NOT_LIVE_REGION',
    'REG_REGION_NOT_WORK_REGION', 'SK_ID_CURR', 'TARGET', 'TOTALAREA_MODE', 'YEARS_BEGINEXPLUATATION_AVG',
    'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_AVG', 'YEARS_BUILD_MEDI',
    'YEARS_BUILD_MODE'
]

plt.figure(figsize=(6.5, 5))

for col in datos_ordenados:
    sns.boxplot(x=application_train[col])
    plt.title(f'Boxplot de {col}')
    plt.xlabel(col)
    plt.show()

Esto se lo hace con la finalidad de analizar las columas evaluadas y determinar si los ourliers hallados pueden ser eliminados

A continuacion, calcularemos los valores faltantes para cada columna, esto con la finalidad de tener una primera idea de las columnas que podremos descartar o filtrar

In [ ]:
total = application_train.isnull().sum().sort_values(ascending = False)
percent = (application_train.isnull().sum()/application_train.isnull().count()*100).sort_values(ascending = False)
missing_application_train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_application_train_data.head(40)

,Total,Percent
COMMONAREA_MEDI,214865,69.872297
COMMONAREA_AVG,214865,69.872297
COMMONAREA_MODE,214865,69.872297
NONLIVINGAPARTMENTS_MODE,213514,69.432963
NONLIVINGAPARTMENTS_AVG,213514,69.432963
NONLIVINGAPARTMENTS_MEDI,213514,69.432963
FONDKAPREMONT_MODE,210295,68.386172
LIVINGAPARTMENTS_MODE,210199,68.354953
LIVINGAPARTMENTS_AVG,210199,68.354953
LIVINGAPARTMENTS_MEDI,210199,68.354953


In [ ]:
umbral = 30
columnas_con_datos_suficientes = missing_application_train_data[missing_application_train_data['Percent'] <= umbral]
columnas_seleccionadas = columnas_con_datos_suficientes.index.tolist()
nuevo_dataframe = application_train[columnas_seleccionadas]
nuevo_dataframe

,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_TYPE_SUITE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,...,DAYS_ID_PUBLISH,DAYS_REGISTRATION,DAYS_EMPLOYED,DAYS_BIRTH,REGION_POPULATION_RELATIVE,NAME_HOUSING_TYPE,NAME_FAMILY_STATUS,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE,SK_ID_CURR
0,0.139376,0.0,0.0,0.0,0.0,0.0,1.0,Unaccompanied,2.0,2.0,...,-2120,-3648.0,-637,-9461,0.018801,House / apartment,Single / not married,Secondary / secondary special,Working,100002
1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Family,1.0,0.0,...,-291,-1186.0,-1188,-16765,0.003541,House / apartment,Married,Higher education,State servant,100003
2,0.729567,0.0,0.0,0.0,0.0,0.0,0.0,Unaccompanied,0.0,0.0,...,-2531,-4260.0,-225,-19046,0.010032,House / apartment,Single / not married,Secondary / secondary special,Working,100004
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unaccompanied,2.0,0.0,...,-2437,-9833.0,-3039,-19005,0.008019,House / apartment,Civil marriage,Secondary / secondary special,Working,100006
4,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Unaccompanied,0.0,0.0,...,-3458,-4311.0,-3038,-19932,0.028663,House / apartment,Single / not married,Secondary / secondary special,Working,100007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unaccompanied,0.0,0.0,...,-1982,-8456.0,-236,-9327,0.032561,With parents,Separated,Secondary / secondary special,Working,456251
307507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unaccompanied,0.0,0.0,...,-4090,-4388.0,365243,-20775,0.025164,House / apartment,Widow,Secondary / secondary special,Pensioner,456252
307508,0.218859,1.0,0.0,0.0,1.0,0.0,1.0,Unaccompanied,6.0,0.0,...,-5150,-6737.0,-7921,-14966,0.005002,House / apartment,Separated,Higher education,Working,456253
307509,0.661024,0.0,0.0,0.0,0.0,0.0,0.0,Unaccompanied,0.0,0.0,...,-931,-2562.0,-4786,-11961,0.005313,House / apartment,Married,Secondary / secondary special,Commercial associate,456254


In [ ]:
column_names = nuevo_dataframe.columns
print(column_names)

Index(['EXT_SOURCE_3', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR', 'NAME_TYPE_SUITE',
       'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'EXT_SOURCE_2',
       'AMT_GOODS_PRICE', 'AMT_ANNUITY', 'CNT_FAM_MEMBERS',
       'DAYS_LAST_PHONE_CHANGE', 'CNT_CHILDREN', 'FLAG_DOCUMENT_8',
       'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_DOCUMENT_2',
       'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5',
       'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_9',
       'FLAG_DOCUMENT_21', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
       'FLAG_OWN_REALTY', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
       'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOC

In [ ]:
total = nuevo_dataframe.isnull().sum().sort_values(ascending = False)
percent = (nuevo_dataframe.isnull().sum()/nuevo_dataframe.isnull().count()*100).sort_values(ascending = False)
missing_application_train_data2  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_application_train_data2.head(20)

,Total,Percent
EXT_SOURCE_3,60965,19.825307
AMT_REQ_CREDIT_BUREAU_HOUR,41519,13.501631
AMT_REQ_CREDIT_BUREAU_DAY,41519,13.501631
AMT_REQ_CREDIT_BUREAU_WEEK,41519,13.501631
AMT_REQ_CREDIT_BUREAU_MON,41519,13.501631
AMT_REQ_CREDIT_BUREAU_QRT,41519,13.501631
AMT_REQ_CREDIT_BUREAU_YEAR,41519,13.501631
NAME_TYPE_SUITE,1292,0.420148
OBS_30_CNT_SOCIAL_CIRCLE,1021,0.332021
DEF_30_CNT_SOCIAL_CIRCLE,1021,0.332021
